In [1]:
import pandas as pd
import numpy as np
import requests
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials())
from pprint import pprint

In [2]:
url_1= 'http://www.acclaimedmusic.net/061024/1948-09art.htm'
artists_1= (pd.read_html(url_1,header=0))[0]["Unnamed: 1"]

pages= np.arange(2,6)
artists_list=[]

for page in pages:
    url_2= f"http://www.acclaimedmusic.net/061024/1948-09art{page}.htm"
    artists_df= (pd.read_html(url_2,header=0))
    artists_list.append(artists_df[0]["Unnamed: 1"])

artists= artists_1.append(artists_list)
artists.dropna(inplace=True)
artists_names= artists.reset_index()
artists_names.drop(artists_names.columns[0], axis=1, inplace=True)
artists_names.rename(columns={"Unnamed: 1": "Artist Name"}, inplace=True)
artists_names

,Artist Name
0,The Beatles
1,The Rolling Stones
2,Bob Dylan
3,David Bowie
4,Led Zeppelin
...,...
995,The Sundays
996,Herbert
997,Stephen Stills
998,Mazzy Star


In [3]:
artists_names["Artist ID"]= ""
artists_names.head()

,Artist Name,Artist ID
0,The Beatles,
1,The Rolling Stones,
2,Bob Dylan,
3,David Bowie,
4,Led Zeppelin,


In [4]:

for index, row in artists_names.iterrows():
    name= row['Artist Name']
    results = sp.search(q='artist:' + name, type='artist')
    
    try:
        artists_names.loc[index,'Artist ID'] = results['artists']["items"][0]["id"]
        artists_names.loc[index,'Artist_Popularity'] = results['artists']["items"][0]["popularity"]      
        artists_names.loc[index,'Artist_Followers'] = results['artists']["items"][0]["followers"]["total"]
        artists_names.loc[index,'Artist Genre']= ",".join(results['artists']["items"][0]["genres"])
        if len(results['artists']["items"][0]["genres"]) == 0:
            artists_names.loc[index,'Main Genre']= np.NaN 
        else:
            artists_names.loc[index,'Main Genre']= results['artists']["items"][0]["genres"][0]
                 
        
    except:
        
        #Printing an explanation message
        print(f"ID not found for {row['Artist Name']}")
        artists_names.loc[index,'Artist Name']= np.NaN
        artists_names.loc[index,'Artist ID'] = np.NaN
        artists_names.loc[index,'Artist_Popularity'] = np.NaN
        artists_names.loc[index,'Artist_Followers']= np.NaN
#         artists_names.loc[index,'Artist Genre']= np.NaN
#         artists_names.loc[index,'Main Genre']= np.NaN
        pass
        
artists_names.dropna(inplace=True)
artists_names




ID not found for Frank Zappa/The Mothers of Invention
ID not found for Janis Joplin/Big Brother and The Holding Company
ID not found for The Buzzcocks
ID not found for Paul McCartney and Wings
ID not found for The Arcade Fire
ID not found for The Small Faces
ID not found for The Dead Kennedys
ID not found for Afrika Bambaataa & The Soul Sonic Force
ID not found for Harold Melvin and The Bluenotes
ID not found for Richard Hell & The Voidoids
ID not found for The Mahavishnu Orchestra
ID not found for The Flamin' Groovies
ID not found for Question Mark and the Mysterians
ID not found for The Nitty Gritty Dirt Band
ID not found for The Art Ensemble of Chicago
ID not found for Rob Base & DJ E-Z Rock
ID not found for PM Dawn
ID not found for Le Mystère des Voix Bulgares
ID not found for Sam the Sham and The Pharaos
ID not found for The Screaming Trees
ID not found for Willie Mae "Big Mama" Thornton
ID not found for Deutsch Amerikanische Freundschaft
ID not found for Michael Hurley/The Unholy

,Artist Name,Artist ID,Artist_Popularity,Artist_Followers,Artist Genre,Main Genre
0,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,90.0,16401883.0,"beatlesque,british invasion,classic rock,merse...",beatlesque
1,The Rolling Stones,22bE4uQ6baNwSHPVcDxLCe,84.0,8675368.0,"album rock,british invasion,classic rock,rock",album rock
2,Bob Dylan,74ASZWbe4lXaubB36ztrGX,79.0,4218024.0,"album rock,classic rock,country rock,folk,folk...",album rock
3,David Bowie,0oSGxfWSnnOXhD2fKuz2Gy,82.0,5521000.0,"album rock,art rock,classic rock,dance rock,gl...",album rock
4,Led Zeppelin,36QJpDe2go2KgaRleHCDTp,82.0,9057038.0,"album rock,classic rock,hard rock,rock",album rock
...,...,...,...,...,...,...
995,The Sundays,4nlqDmbzFzbNITaqjJv7D7,52.0,86621.0,"alternative pop,alternative rock,britpop,dream...",alternative pop
996,Herbert,5zCaQxjl110XTrm4LQ1CxY,70.0,100741.0,"classical,classical performance,orchestral per...",classical
997,Stephen Stills,4WlSvDKaq1PA2Nr7cCIPxX,56.0,163207.0,"blues rock,classic rock,country rock,folk,folk...",blues rock
998,Mazzy Star,37w38cCSGgKLdayTRjna4W,64.0,410363.0,"alternative rock,dream pop,melancholia,slow core",alternative rock


In [5]:
artists_names.count()

Artist Name          947
Artist ID            947
Artist_Popularity    947
Artist_Followers     947
Artist Genre         947
Main Genre           947
dtype: int64

In [6]:
top10_tracks= pd.DataFrame(columns=["Artist","Artist ID","Tracks","Track ID","Release Date","Track Duration","Popularity"])
top10_tracks


,Artist,Artist ID,Tracks,Track ID,Release Date,Track Duration,Popularity


In [7]:

counter=1
for index, row in artists_names.iterrows():
    artist_id= row['Artist ID']
    artist_top_tracks = sp.artist_top_tracks(artist_id)

     
    track = artist_top_tracks['tracks'][:10]

    for tracks in range(len(track)):    
        try:
            top10_tracks.loc[counter,'Tracks'] = track[tracks]["name"]
            top10_tracks.loc[counter,'Artist'] = track[tracks]["album"]["artists"][0]["name"]
            top10_tracks.loc[counter,'Artist ID'] = track[tracks]["album"]["artists"][0]["id"]
            top10_tracks.loc[counter,'Track ID'] = track[tracks]["id"]
            top10_tracks.loc[counter,'Release Date'] = track[tracks]['album']['release_date']
            top10_tracks.loc[counter,'Track Duration'] = track[tracks]['duration_ms']
            top10_tracks.loc[counter,'Popularity'] = track[tracks]['popularity']
            
            counter+=1
        
        except:
            
            pass
    
top10_tracks    


,Artist,Artist ID,Tracks,Track ID,Release Date,Track Duration,Popularity
1,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Here Comes The Sun - Remastered 2009,6dGnYIeXmHdcikdzNNDMm2,1969-09-26,185733,83
2,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Come Together - Remastered 2009,2EqlS6tkEnglzr7tkKAAYD,1969-09-26,259946,78
3,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Let It Be - Remastered 2009,7iN1s7xHE4ifF5povM6A48,1970-05-08,243026,78
4,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Yesterday - Remastered 2009,3BQHpFgAp4l80e1XslIjNI,1965-08-06,125666,77
5,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Hey Jude - Remastered 2015,0aym2LBJBk9DAYuHHutrIl,2000-11-13,425653,76
...,...,...,...,...,...,...,...
9424,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Future Blues,7cAXWkbpR8Szb06AFXhAmJ,2013-08-27,180773,15
9425,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Match Box Blues,4dBUAjMjmAednMqt8PU4AI,1992-01-01,187400,14
9426,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,Rabbit Foot Blues,69xrgBndFd3zs6kqVZ8LDO,1991,179095,13
9427,Blind Lemon Jefferson,6EsuMifB8c5rm5P2DdjiLe,That Black Snake Moan,0I4nvtM87RCvywkmeinsqI,1992-01-01,178640,12


In [8]:
for index, row in top10_tracks.iterrows():
    try:
        track_id = row["Track ID"]
        track_data = sp.audio_features(track_id)
        top10_tracks.loc[index,"Danceability"] = track_data[0]['danceability']
        print(f"{index} {row['Track ID']}")
    except:
        top10_tracks.loc[index,"Danceability"]=np.NaN
        pass
top10_tracks.head(20)



1 6dGnYIeXmHdcikdzNNDMm2
2 2EqlS6tkEnglzr7tkKAAYD
3 7iN1s7xHE4ifF5povM6A48
4 3BQHpFgAp4l80e1XslIjNI
5 0aym2LBJBk9DAYuHHutrIl
6 5jgFfDIR6FR0gvlA56Nakr
7 4pbG9SUmWIvsROVLF0zF9s
8 5ZBeML7Lf3FMEVviTyvi8l
9 3KfbEIOC7YIv90FIfNSZpo
10 7DD7eSuYSC5xk2ArU62esN
11 63T7DJ1AFDD6Bn8VzG6JE8
12 7HKez549fwJQDzx3zLjHKC
13 2PzU4IB8Dr6mxV3lHuaG34
14 6H3kDe7CGoWYBabAeVWGiD
15 1Ud6moTC0KyXMq1Oxfien0
16 77oU2rjC5XbjQfNe3bD6so
17 52dm9op3rbfAkc1LGXgipW
18 1GcVa4jFySlun4jLSuMhiq
19 61UuPxxYUvacEH6SHIK3sU
20 2aQHcpFfrUoTc9CtxPGotZ
21 6HSXNV0b4M4cLJ7ljgVVeh
22 3AhXZa8sUQht0UEdBJgpGc
23 52vA3CYKZqZVdQnzRrdZt6
24 1lqMLr9Wj7SM2F9AikGcxN
25 18GiV1BaXzPVYpp9rmOg0E
26 4K1imZQQ0yKtJ40vGmUajS
27 2WOjLF83vqjit2Zh4B69V3
28 6Vcwr9tb3ZLO63F8DL8cqu
29 3RkQ3UwOyPqpIiIvGVewuU
30 3y4Uza6K58JXQ7RYya8ZI5
31 7Jh1bpe76CNTCgdgAdBw4Z
32 2EC9IJj7g0mN1Q5VrZkiYY
33 0pQskrTITgmCMyr85tb9qq
34 72Z17vmmeQKAg8bptWvpVG
35 3ZE3wv8V3w2T2f7nOCjV0N
36 3ix6K4wZY29bCujrSznwFZ
37 3gxEZXUjrNbl3TlSrTGbR5
38 6mib3N4E8PZHAGQ3xy7bho
39 0LrwgdLsFaWh9VXIjB

312 3whRKAOlJ0M3banzcChvQv
313 6LzjgzpVl0hB3YrBU1b8Z1
314 5U3i59kbTLrxo46TU1FRnF
315 2ivdsb1RcEJjIs5tuHkUC9
316 47Zqlw3jmibtVPy8usPjBj
317 5OcyX5eg7VrJuI833qOCUc
318 44AgaeLQCnvk3PPHIaDeDk
319 00pGV4EInVd77cnOIwPTCv
320 2J1IYUHS1XXyd2NepfTlco
321 29SyMC0plk6qw8NMF7lfRL
322 0UAJH0k4k3slcE83a9UGCe
323 78JmElAFmrPNhLjovDR9Jm
324 0DQqzCHlPyKQXfF8uL01SI
325 5nrmGFJ87crVoJF5xdRqwn
326 7obb4s6A7gf0Lc2AGxodMy
327 7DTfOX4SIuMMt33sSVvLRr
328 1LjPCQHkAYnJbQUgSoHaye
329 5e3isD5st7PGYzSJuoRSIV
330 5HOQNAXcPSwV7Gt8AV8qiP
331 0Puj4YlTm6xNzDDADXHMI9
332 5NLuC70kZQv8q34QyQa1DP
333 5fpizYGbi5IQoEraj6FP0R
334 5qxChyzKLEyoPJ5qGrdurN
335 2tY1gxCKslfXLFpFofYmJQ
336 3gKwVWwKmeuFtPubICbOGc
337 5h40ovzXu7rF7Giwb9oXYy
338 0gmGBCJ5XhOmoNR37MmxEE
339 21REQ1bCUWphT2QK3bLWYQ
340 56H98l4uinRifnTH8e308N
341 4BP3uh0hFLFRb5cjsgLqDh
342 5DnT9a5IM3eMjKgXTWVJvi
343 20OFwXhEXf12DzwXmaV7fj
344 2gE95JskwQ1pCACTpGe1Db
345 36gPq8WG7tDxrblyGVUCiT
346 3RTUfvRgq3pYTf7XY0FIcS
347 2EZc13S7YRi6bTS3SKteRN
348 11DjZQEZ69EXLo77HVj6yW
3

618 5p95DUZplmVxY0XCQ3hxUV
619 47vmcuvMWFIsMaiHFIGSIu
620 1Rph9i0FWKDa9Aq9H4iiMx
621 0OMNQyneWmmZtTQpULYJcl
622 5G9LvzXcBoIBXOd2jzdJTs
623 3vEmpJQC2p8h0U3OiBufQm
624 5OnyZ56HLhrWOXdzeETqLk
625 4z1fNs2B7KndCsvyPgrhq5
626 0X0LTI0MQG9S3FLSUvjrj5
627 3rrtbnOBDzKQfyYKTezEFe
628 080thMRVMSfE5fem7HBaoF
629 20ZvzoDSefcZo6bj10jgGC
630 0mc51xomEC6CZUZdB8xgQU
631 67Hna13dNDkZvBpTXRIaOJ
632 0j5FJJOmmnXPd0XajFWkMF
633 7uv632EkfwYhXoqf8rhYrg
634 2BndJYJQ17UcEeUFJP5JmY
635 3kKVqFF4pv4EXeQe428zl2
636 7xpw5FSNYXKfdQSBuO0THv
637 46bTgJAQkrkNRxMuFnpRyD
638 1vNNfTgHsrpOXeiaXQBlH7
639 1Rezzt36ybaT2ZbDZpv83D
640 3N2UhXZI4Gf64Ku3cCjz2g
641 1qTfHUVPsZTnrmvwQrAQdW
642 3AXDLGI5TIYi72JcYFvNpv
643 23xk9Rf7oIHVUU1JvmXYFn
644 0aD8W0ChzDgEKpB1zsJ8zv
645 4WYFQLVqla6tpYtKsSqcgu
646 3uqEiaO6MUoM89nLSIs05c
647 5jfgilRtwvpSfZyKkqIGNW
648 6NWD6siQErobvmydHcmhCw
649 1iesv8xiE4cgtssvBUBM4K
650 5krjye2PUpiGedzdbaqWcz
651 2QfiRTz5Yc8DdShCxG1tB2
652 6FT83pFXKhDlXDsNJFAHWz
653 3qt3ybU6MScMxMKL2TYBvt
654 6C7aTTCUWRK7dD379yUT3W
6

923 1OE5l6sedVcIFELMuxQOPI
924 7M7AwtGvWdMYudqx5Iuh1m
925 3euDGpS2R0NC2Xssqxohva
926 1I6l01BaBgBCVkfQCpIlR6
927 1lNRVjK8MukRZpeurYssIx
928 0mJQlCl9YgxW7kyeltNiVk
929 4DpBfWl3q8e0gGB76lAaox
930 7x2xjJV3YAPeLQJ7u3Kjet
931 16qzGrIMWoxerw2gnW0zuv
932 3EtIraJEHVSbBvLw5msioH
933 0hNVjU6JKydHts0SAjHCno
934 1drLQNS9D5z9lBP4DbtO2R
935 2YkQuh1bijWzQJie9KfFQi
936 3Uh7rAb7F0XGVpEEDwfH1k
937 4XFZey4zmgQV551M0hfaUg
938 4lGUA1a5NCgoRVrvPCrw9G
939 6d01Jkzt0zYiQINWJJFCaf
940 42yexCY4dCftowtAZXuAIj
941 745H5CctFr12Mo7cqa1BMH
942 7MiLmLbwNoyf47xQ4TCVYp
943 4CoGNqLap7UGU5Q3VdKug0
944 676Rf0qIOh2gKNEl9TnCOx
945 3496rr5XSGD6n1Z1OKXovb
946 39Bd345OWEhRNyfayhp9gv
947 6aL6pPBaTMGgM558mplSnj
948 1tjpccwiGZTMDfHr5VGCmT
949 3Rhxe6P1im82NFVkt9X2qZ
950 3WuFR7U2ZgLXaHYjaMJjtu
951 2SWn864dJI5RNyrp3F4Kp1
952 2yzPBlE5UXK2sqvnNM9QQ0
953 6IDHh3FcwLVcG5NsmpQgg2
954 33gDIKlrUnY1dTWYsm5BFL
955 6M0HMy7UhtlPYTlpaW4qRk
956 7KeoeLwqtTIIgeHkvgxkeT
957 6SoXgLYqIDiR7Q4Npijwnp
958 4koGF4vd2AV8pF7R3TVBVq
959 18fr7t1FKDUiOg8DXxiZwL
9

1219 32PeRx8ulVvH9kpsIRp7L7
1220 38An8NSITHa1c2NI7TmYAD
1221 3HfEgAaf0koxBpBB8NvGda
1222 52UcjsM15hjCQAUbTW2hy1
1223 4O1j2o3XaZQ1H8xwibgc4q
1224 5KBKqxYY263Tr0haAu3fMz
1225 7ITRNB9OckYIp2SW2iXU8U
1226 41R4fI1Z1w2gsJEFQWdeJs
1227 1kAQmY7yNW6LFdDftDbe1X
1228 0oiGUMBeL8Bn0csxAl4a1W
1229 5NziFO1dYnQPu2EXdTB4UL
1230 1I63q5aYjoyai9CI8QXDrn
1231 4uGIJG1jYFonGc4LGp5uQL
1232 4vpeKl0vMGdAXpZiQB2Dtd
1233 0yLfXULLuRtVj2L4DIrwO7
1234 2zbGXr2XCcRR40R7ymEENc
1235 5eGKna0aflc9wmMoDcIja4
1236 04lTbtPmiRzwsxZRs2sdk9
1237 0TBntp6t4oS6UXjxikO5n7
1238 7GeQk8PqG5NAENxCRLcw0B
1239 0dWRLm1poqzSYI5FsV6Bqo
1240 0Mlt9aKcyMpkm28EjKrwQR
1241 2hitsKa8SthKhRJBXUHbIv
1242 0ToHhkK4qtwEyKOxhQpMbJ
1243 29HaKOpeLSYvqdFyEQSRdj
1244 0gCJkpD0RhF2rjiALLHBQL
1245 4bjvLvKovcWqZwDbXT5QQX
1246 3gQMqeXqiHFCafaTtFnHF8
1247 2iqKVzMZpyUVPwxIVaQkhm
1248 40WeJU3odsbq1fXNbub4nh
1249 2jZPEeW2ynykb2GkVqpOkJ
1250 3O4VF8UYD6CCuhj6d4TQPa
1251 3BQLTtpQOYGXQY2lT3ZZeU
1252 1B5kQto4k99RG67TJEwwS5
1253 731zWsKI2KvdBPO5MWdjSy
1254 7fwUb4zqdjakeey

...

8300 5ehrJB4xFea5kph6jVoWtS
8301 2gdX6cR2i2KlXdYxG3lNfw
8302 7awYVgOAmDqQIswnzClZDC
8303 2ZJZcqM60OwgqTO1OvJpHL
8304 7GFZ5xehLR65KbD9koUc4f
8305 63qZainb4qJfABhv3EOhzX
8306 7zCI47ILA6MAYXrlI487Hs
8307 2MVG3i3yWBhG0DY2VBqOVh
8308 4EMnfH9SkLEbLfHMbrFssK
8309 1eamsmwcYYhJwTgMFdQ6YN
8310 4y7YMKVaXwOB7F6tyVLrF9
8311 0gGn0vQaOlFFlan4h8vWz1
8312 6X542A49pirp8Y511KDH6y
8313 0uVGDPBHf8WvWhRsQdRYle
8314 66M9J2RW16TZOgmfPNvBkU
8315 1DWJmMRfAWYbqasrfW5wAd
8316 4I9r7EvB9284UPV0gD43bF
8317 6OhUzO1QhRdKs1PVZh5buG
8318 2jIRiLmzqC45L0SnOMimHV
8319 06TuMCjDEYjTOJScCRzmjf
8320 3R97rNX7JnmshCWBwOSFet
8321 3ZqjDpr3qCZnZBPcd0Hpj2
8322 2vG1yrWSMiL6egg6w4e9ma
8323 4htbAEZWr53J08x3dUv00W
8324 1fHgO6v34KSSBKrNQE6q21
8325 1FLVjfvrLzIwF0nI8OXhJw
8326 5AYyDRotbApDiucOpJnFGr
8327 53sFlnEQbHADehMkqumk96
8328 61vIGLnOsYaydUUKfJ9XC5
8329 2JdzB67NvIa90K4gEZPLeX
8330 3VEFybccRTeWSZRkJxDuNR
8331 7wck5ORA8dJB8To9rc10gi
8332 4WamMWxFi6ts3xkIhivF5c
8333 7hpbjF2BGRPPIdyrzX5tZA
8334 7bCKBqdYJI8fvFH48sT2IN
8335 24obGxQaI9mRyH6

8593 73lCAVSmr6nJ9HjKm4oiYn
8594 65brr7OzgyekSTmhLpBn4H
8595 5tsbWKd0BvFPjUJW2yUgVS
8596 1iSNYPWKbcUrcANG5Fcuba
8597 1TfxKIJTBpfEl7BS30hYbu
8598 1NiFAfPjJukUceg0etgxGC
8599 4Q4w8aRdfhIIEhBututktL
8600 4CsTkNvkwQLel6sxQqrXSj
8601 3hKScGxI28Zor6zJ7JL7aS
8602 1Lpmme6uCsPJIdcbR5aePx
8603 5eGDKrTYl2h6ponZyMMhgZ
8604 3d5Cnswx1dxmjUbn1Dp3JQ
8605 7tZ1sF5HLhqtRO0bITs4lV
8606 6Y19VBGJBoixg6umIkE7ar
8607 4UQXPGa1zyIOsjVP5lhI52
8608 09NRBlKCJmHPedMcOeOz1Y
8609 4B2lJinAkeNLSJjcq3dg8Q
8610 5mb6SzBnxv1ywFSH9V3uxd
8611 3JiiqMYLbnHa1qys1JOpFK
8612 08YEGpKt2LHJ0URCXKHEie
8613 7FCYixd46BlSiO2memrsPo
8614 7w7X5rwDFXwHSqlxtsljP7
8615 3BFXgZr628FqwDP3pQCgvk
8616 3AkxSspcYOvhWTkaMvqyaD
8617 6sxBLLKfLjVaTg5KE4Cy75
8618 4CVP2pvqUxH9tExsHJnzV9
8619 0F845nujLVqCb0XMZCh5Pc
8620 4l9hml2UCnxoNI3yCdL1BW
8621 06UkYE0SZ3Y8zGRFOuWGAe
8622 4gFlHAye7To29Fs4UszQUH
8623 1QlTNF43OyKK9BorIRsJ8o
8624 2d64G7VaZdHQuAquz5HQNu
8625 1Zp7ySmnsdP0qrr1plNIqP
8626 0GExhcihMVjrKsad725W62
8627 3pfbQmD5xGH9Tcc7GqGfSL
8628 05lOXZuJyELJT0g

8887 4YWituEPTLy2KZgDTXZUut
8888 2Pjb1ApkiRtEmAGAmPxUnv
8889 1a2iF9XymafjRk56q7oCxo
8890 5XNpdKmlLJPUbwKQceX2tW
8891 5MtN38MGEWJt60LwtBmFNP
8892 0nFZ2HQRnIxiwraCm2X6Uq
8893 7zto61V8ySp03Qi6X1LU2X
8894 1kY6OOTBmRNzhYmu9xSH3a
8895 5MVMARaMoclifNmBtPu0dD
8896 0h4jHdNzUrV9eGR2PT5R8c
8897 1cyftogE58xHzSgb77zcsn
8898 3QwDN08fGnGfRrsn5wLTej
8899 620zXJyl50tD6FMobTqZN6
8900 5JqCu72Whv6PWK8rcyHhVR
8901 5VZENOCI4fqli2PTmtYfbf
8902 2ImCB7nr7wdIjmKljOCx7U
8903 1bfD0QqQ5O4yWl4lB8zp7m
8904 0EzjclFYdLpKhTFwgqUKQ9
8905 65Ca5DXSwq2kW0hk6q6Ieu
8906 6Xwh66Zom2mcNu96takcnR
8907 7j1s1GSJJbBN7qQ2O9IA1F
8908 0kxzulga33T76Mmkg6wydz
8909 5TvcggVu3s2P1fMAS8BupX
8910 6IXANTg9CWP73yEXqTHkBx
8911 5TjDNPyTBGM2XcsYL7NEaQ
8912 3elP66qkaTnPpnLDTrVHik
8913 3GheKYtjDLMWwiNhLptm8l
8914 6dAffqfVEYUmntU7K9QtlA
8915 60ffP4txpmfWL3v3djl2H9
8916 2iGqxsVkGdF7ooa6ms577F
8917 5bc1kaUqSKCmSJ75HoCOgz
8918 2ZkGTlqgWtX2EOAV1uAyjG
8919 3Fg4SlOiqKDodVJbAM2Gut
8920 5iDmsAvKVxhk6GDw3OGrj6
8921 2YOUJ7PjvrJxrg9dL1PpT3
8922 26lgpX6SguLbcah

9182 6gpai6xL08vD5mQ4fXTwxL
9183 5MRRZxscYnbYKHvOQdxtdw
9184 4m1BHxinyAOrymQFKqTsr6
9185 7LZwFnHW2oWzFqtsmfrWw7
9186 1B45KRIxXBnKosy2OGvPEJ
9187 4oxqp216awjiyQYwndUI8A
9188 0vl6IBNXnbrMakBQa4Le4r
9189 1bYItjcPRtxZs6Ir71O5RB
9190 7yED4n2U8RR5LKZVmisiev
9191 4AyULNTWuCtbpxzY2qVowJ
9192 0UuRIovHyU6KeTzY4gS0L2
9193 3Cq4BUlxDg4ulj7XnN2COP
9194 4tqGZzY8YmngfaTqUAHENj
9195 4BH8EuPAxeFAh1rSWtnGdD
9196 3aNVmx1plEeXKApSzTvUTc
9197 6ZsKRsnPsdZvXfqqE6bTPy
9198 1c4oqyJGptLz2Yt7LLheX8
9199 4dc1G54hSmSL5HQuQvRNA1
9200 4vx5PxhKZ8GBnB8MR3mpry
9201 6WKqmLoDX2wqswwCEBCOkf
9202 6Gg5JxtMkKATN7KxhBfBlF
9203 3Jw64HImlMixvjPXo5tRVP
9204 4x8iJUW5a3qTsBLqboiwur
9205 7546NLQYw3yoXIBbWnSUwN
9206 5BYYSsWgCfiuH0d1wHFGAF
9207 1Pr6oNpETiFZdZYZd5WtNw
9208 7xTRgPWk48fIuoBoyl7ro6
9209 3c6wivu78aLN7ZXp8UbpaI
9210 56K7H3Ipsce0Csn2v4u8kZ
9211 0Ya5bHuA3kyiePOhIQfQtW
9212 7L5FEYVfpIEGHJOv6gZ6EW
9213 0WON7ccM42foBwZikHsf7v
9214 4ML8gLVDid1m2pRDHGUQQ7
9215 5wUA6MSNGGORassu6tHnAJ
9216 00lcn90nnKszEEuVgO8VzK
9217 5STlvzO9CX7G8xd

,Artist,Artist ID,Tracks,Track ID,Release Date,Track Duration,Popularity,Danceability
1,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Here Comes The Sun - Remastered 2009,6dGnYIeXmHdcikdzNNDMm2,1969-09-26,185733,83,0.557
2,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Come Together - Remastered 2009,2EqlS6tkEnglzr7tkKAAYD,1969-09-26,259946,78,0.533
3,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Let It Be - Remastered 2009,7iN1s7xHE4ifF5povM6A48,1970-05-08,243026,78,0.443
4,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Yesterday - Remastered 2009,3BQHpFgAp4l80e1XslIjNI,1965-08-06,125666,77,0.332
5,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Hey Jude - Remastered 2015,0aym2LBJBk9DAYuHHutrIl,2000-11-13,425653,76,0.386
6,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Blackbird - Remastered 2009,5jgFfDIR6FR0gvlA56Nakr,1968-11-22,138386,75,0.686
7,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,I Want To Hold Your Hand - Remastered 2015,4pbG9SUmWIvsROVLF0zF9s,2000-11-13,145746,74,0.490
8,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Twist And Shout - Remastered 2009,5ZBeML7Lf3FMEVviTyvi8l,1963-03-22,155226,74,0.482
9,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,In My Life - Remastered 2009,3KfbEIOC7YIv90FIfNSZpo,1965-12-03,146333,73,0.688
10,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,Help! - Remastered 2009,7DD7eSuYSC5xk2ArU62esN,1965-08-06,139560,72,0.534


In [11]:
grouped= top10_tracks.groupby("Artist")
count= grouped["Track ID"].count()
count

Artist
!!!                                             10
$NOT                                             1
...And You Will Know Us by the Trail of Dead    10
10cc                                            10
2Pac                                            10
                                                ..
Zach Williams                                    1
case/lang/veirs                                  7
dvsn                                             1
for KING & COUNTRY                               1
k.d. lang                                        5
Name: Track ID, Length: 1136, dtype: int64

In [ ]:
top10_tracks